In [20]:
import torch.nn as nn

class VGG_16(nn.Module):
    def __init__(self):
        super(VGG_16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
           
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(2, 2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2, 2)
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2, 2)
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2, 2)
        )

        self.conv = nn.Sequential(
            self.layer1,
            self.layer2,
            self.layer3,
            self.layer4,
            self.layer5
        )
        
        self.linear = nn.Sequential(
            nn.Linear(in_features=512, out_features=512),
            nn.ReLU(),
            nn.Dropout(0),

            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Dropout(0),
            
            nn.Linear(256,10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 512)
        x = self.linear(x)
        return x


In [21]:
# train
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 数据加载和预处理
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

train_loader = DataLoader(
    datasets.CIFAR10('./data/CIFAR10', train=True, download=True, transform=transforms_train),
    batch_size=64, shuffle=True)

# 模型实例化和优化器设置
model = VGG_16()
optimizer = optim.Adamax(model.parameters(), lr=0.01)

# 设备设置
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

# 训练过程
for epoch in range(1, 30):
    for batch_idx, (data, target) in enumerate(train_loader):
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 保存模型
torch.save(model.state_dict(), f'./model/cifar10_vgg.pt')



Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.348259
Train Epoch: 1 [640/50000 (1%)]	Loss: 2.445001
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.364627
Train Epoch: 1 [1920/50000 (4%)]	Loss: 2.307438
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.294033
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.286596
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.303299
Train Epoch: 1 [4480/50000 (9%)]	Loss: 2.297231
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.304354
Train Epoch: 1 [5760/50000 (12%)]	Loss: 2.267999
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.318452
Train Epoch: 1 [7040/50000 (14%)]	Loss: 2.271614
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.295196
Train Epoch: 1 [8320/50000 (17%)]	Loss: 2.289468
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.288243
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.328173
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.320714
Train Epoch: 1 [10880/50000 (22%)]	Loss: 2.270828
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.268077
Train Epoch: 1 [12160/50000 (24%)]	Loss:

In [23]:
# test.py
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# 数据加载
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

test_loader = DataLoader(
    datasets.CIFAR10('./data/CIFAR10', train=False, download=True, transform=transforms_train),
    batch_size=1000, shuffle=True)

model = VGG_16()
model.load_state_dict(torch.load('./model/cifar10_vgg.pt'))

model.to(device)
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')




Files already downloaded and verified

Test set: Average loss: 0.4861, Accuracy: 8843/10000 (88%)

